### 1.requests请求百度API，获取POIs

#### API请求的URL地址
矩形区域检索 http://api.map.baidu.com/place/v2/search?query=银行&bounds=39.915,116.404,39.975,116.414&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&page_size=20&page_num=1 //GET请求

In [1]:
import pandas as pd
import requests 
import json
import time

In [34]:
url = 'http://api.map.baidu.com/place/v2/search?query=银行&bounds=39.915,116.404,39.975,116.414&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&page_size=20&page_num=0'
res = requests.get(url).text
json_obj = json.loads(res)
print(json_obj['results'][-1])

{'name': '中国工商银行24小时自助银行(安德里北街店)', 'location': {'lat': 39.963383, 'lng': 116.408833}, 'address': '北京市东城区安德里北街18', 'province': '北京市', 'city': '北京市', 'area': '东城区', 'street_id': '0962c342b60e0ab39fec2b5c', 'detail': 1, 'uid': '0962c342b60e0ab39fec2b5c'}


In [35]:
def get_data(json_obj):    
    for item in json_obj['results']:
        js = {}
        js['name'] = item['name']
        js['lat'] = item['location']['lat']
        js['lng'] = item['location']['lng']
        yield js

In [37]:
pd.DataFrame(get_data(json_obj)).head()

,name,lat,lng
0,中国银行(安定门外支行),39.958051,116.413457
1,中国工商银行(北京东华门支行),39.921373,116.411402
2,交通银行(北京和平里支行),39.968787,116.410049
3,中国工商银行(北京和平北路东口支行),39.968629,116.410245
4,北京银行(中轴路支行),39.957382,116.404191


In [4]:
class Baidupoi(object):
    def __init__(self,query,bounds):
        self.query = query
        self.bounds = bounds
        
    def get_urls(self):
        urls = []
        for i in range(20):
            url = 'http://api.map.baidu.com/place/v2/search?query={}&bounds={}&output=json&\
ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&page_size=20&page_num={}'.format(self.query,self.bounds,i)
            urls.append(url)
        return urls

    def get_data(self):
        for i,url in enumerate(self.get_urls()):
            try:
                print(i,url)
                js_obj = requests.get(url).text
                data = json.loads(js_obj)
                if data['total'] != 0 :
                    for item in data['results']:
                        js = {}
                        js['name'] = item['name']
                        js['lat'] = item['location']['lat']
                        js['lng'] = item['location']['lng']
                        yield js
                else:
                    print('The data all has been processed!')
                    break
                with open(r'../../file/subject/chapter5/log.txt','a') as file:
                    file.write(time.ctime() + ' status:done'+'\n'+ url + '\n')
            except:
                print('error!')
                with open(r'../../file/subject/chapter5/log.txt','a') as file:
                    file.write(time.ctime() + ' status:error' +'\n' + url + '\n')
                
if __name__ == '__main__':
    bounds = '34.32016,108.962152,34.328478,108.972357'
    for query in ['餐饮','小区']:
        par = Baidupoi(query,bounds)
        print(query)
        dt = par.get_data()
        print(pd.DataFrame(dt)) 
    print('done')

餐饮
0 http://api.map.baidu.com/place/v2/search?query=餐饮&bounds=34.32016,108.962152,34.328478,108.972357&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&page_size=20&page_num=0
1 http://api.map.baidu.com/place/v2/search?query=餐饮&bounds=34.32016,108.962152,34.328478,108.972357&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&page_size=20&page_num=1
2 http://api.map.baidu.com/place/v2/search?query=餐饮&bounds=34.32016,108.962152,34.328478,108.972357&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&page_size=20&page_num=2
3 http://api.map.baidu.com/place/v2/search?query=餐饮&bounds=34.32016,108.962152,34.328478,108.972357&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&page_size=20&page_num=3
4 http://api.map.baidu.com/place/v2/search?query=餐饮&bounds=34.32016,108.962152,34.328478,108.972357&output=json&ak=VYt6ipe4cG9t1CpURQr58ulXep2pcpY6&page_size=20&page_num=4
5 http://api.map.baidu.com/place/v2/search?query=餐饮&bounds=34.32016,108.962152,34.328478,108.972357&output=json&ak=VYt6ipe4cG9t1CpURQr58u

In [14]:
time.ctime()

'Fri Jun  5 22:53:19 2020'